<a href="https://colab.research.google.com/github/awal015/CapstoneStep4/blob/main/CF_Practice_3_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
rounakbanik_the_movies_dataset_path = kagglehub.dataset_download('rounakbanik/the-movies-dataset')
threnjen_board_games_database_from_boardgamegeek_path = kagglehub.dataset_download('threnjen/board-games-database-from-boardgamegeek')
benhermangamingcom_game_ratings_from_one_user_path = kagglehub.dataset_download('benhermangamingcom/game-ratings-from-one-user')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

from fastai.collab import *
from fastai.tabular.all import *


def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [ ]:
ratingsImported = pd.read_csv('../input/board-games-database-from-boardgamegeek/user_ratings.csv')
ratingsImported = ratingsImported.sample(n=1000000)
singleRatingsImported = pd.read_csv('../input/Game-Ratings-From-One-User/Game Ratings Form 1 User.csv')
BGGId = pd.read_csv('../input/board-games-database-from-boardgamegeek/games.csv', usecols=(0,1)).drop_duplicates("Name")
descriptions = pd.read_csv('../input/board-games-database-from-boardgamegeek/games.csv', usecols=(1,2)).drop_duplicates("Name")

In [ ]:
def getUserData():
    array = pd.DataFrame({"Username":[], "Name":[], "Rating":[]})
    while input("do you want to input more data (y/n): ") == "y":
        gameName = None
        while (not gameName in set(BGGId['Name'])):
            if gameName != None:
                print("could not find that game on the list, please try again")
            gameName = input("please input the name of the game you want to rate: ")
        gameRating  = int(input("please input the rating of the game 1-10: "))
        array = array.append({"Username":"OurUser", "Name":gameName, "Rating":gameRating}, ignore_index=True)
    return array

In [ ]:
singleRatings = getUserData()
singleRatings = singleRatings.merge(BGGId)
singleRatings = swap_columns(singleRatings, 'BGGId', 'Name')
singleRatings

In [ ]:
singleRatings = singleRatingsImported.copy()
singleRatings = singleRatings.rename(columns={'Game':'Name'})
singleRatings = singleRatings.rename(columns={'Happy Rating':'Rating'})
singleRatings = singleRatings.merge(BGGId)

singleRatings = swap_columns(singleRatings, 'BGGId', 'Name')

singleRatings = singleRatings.rename(columns={'Moody Rating':'Username'})
singleRatings['Username'] = 'OurUser'
singleRatings['Rating'] = singleRatings['Rating'].astype(float)

singleRatings = swap_columns(singleRatings, 'Username', 'BGGId')
singleRatings = swap_columns(singleRatings, 'Rating', 'BGGId')


ratings = ratingsImported.copy()
ratings = ratings.merge(BGGId)

ratings = swap_columns(ratings, 'Username', 'BGGId')
ratings = swap_columns(ratings, 'Rating', 'BGGId')


size = ratings.groupby(ratings['Username'], as_index=False).size()
ratings = ratings.merge(size)

m = ratings['size'].quantile(0.95)
#m = 6
ratings = ratings.loc[ratings['size'] > m]

ratings = ratings.drop(['size'], axis=1)

ratings = singleRatings.append(ratings, ignore_index=True)


labels, uniques = pd.factorize(ratings['Username'])
usernameToUserID = pd.DataFrame(data = ratings['Username'].copy())
usernameToUserID['UserID'] = labels
usernameToUserID = usernameToUserID.drop_duplicates()

ratings['Username'] = labels
ratings = ratings.rename(columns={'Username':'UserID'})

In [ ]:
m

In [ ]:
ratings

In [ ]:
dls = CollabDataLoaders.from_df(ratings, item_name='Name', bs=64)
learn = collab_learner(dls, n_factors=20, y_range=(0, 13))

#The function below is based heavily on the code from Dhruv Metha, shown at
#https://forums.fast.ai/t/collaborative-filtering-predict-inference-part-1-2020/90240/3
def get_rating(learn, user):
    if type(user) == str:
        user = usernameToUserID[usernameToUserID['Username'] == 'OurUser']['UserID'][0].item()
    with torch.no_grad():
        games = torch.arange(learn.model.i_weight.num_embeddings)
        users = tensor([user])
        try:
            dot = learn.u_weight(users).unsqueeze(1) * learn.i_weight(games)
            res = dot.sum(-1) + learn.u_bias(users) + learn.i_bias(games).squeeze()
            return (torch.sigmoid(res) * (learn.y_range[1]-learn.y_range[0]) + learn.y_range[0])[0]
        except:
            print('User not recognized')

In [ ]:
learn.fit_one_cycle(3, 5e-3, wd=0.1)
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['Name'][i] for i in idxs]

In [ ]:
learn.recorder.plot_loss()

In [ ]:
def printPredictions(user):
    def printIndexs(indexs, printCount = 20):
        print('----------------------------------------------------------------------------------------------------')
        i = 0
        while i < printCount:
            index = indexs[i]
            if (dls.classes['Name'][index] in singleRatings["Name"]):
                continue
            i+=1
            print('')
            print(" ",
                  dls.classes['Name'][index], ":",
                  math.floor(predictions.numpy()[index]*10)/10
                 )
            print(descriptions.loc[descriptions['Name'] == dls.classes['Name'][index]]['Description'])
            print('----------------------------------------------------------------------------------------------------')
    predictions = get_rating(learn, user)
    indexs = predictions.argsort(descending=True)
    print("I think you will like:")
    printIndexs(indexs, 10)
    print("")
    indexs = predictions.argsort(descending=False)
    print("I dont think you will like:")
    printIndexs(indexs, 10)

In [ ]:
printPredictions(0)